In [3]:
%run SharedParams.ipynb
%run MerkleTree.ipynb
%run Main.ipynb


In [38]:
class Fri:
    
    def __init__(self, cp_eval, fiat_shamir, proof_data):
        assert len(cp_eval) == g_order        
        self.layers = [MerkleTree(cp_eval)]
        while True:
            proof_data.append(f"layer_root:{self.layers[-1].root}")
            beta = fiat_shamir.get_random_int(proof_data)
            new_layer_eval = self.next_eval(self.layers[-1].data, beta)
            # print(PR.lagrange_polynomial(zip([w ** int(len(cp_eval)/len(new_layer_eval)) * g ** i for i in range(0, len(cp_eval), int(len(cp_eval)/len(new_layer_eval)))], new_layer_eval)))
            if len(new_layer_eval) > RS_factor:
                self.layers.append(MerkleTree(new_layer_eval))
            else:
                self.last_layer_const = new_layer_eval[0]
                assert all(self.last_layer_const == val for val in new_layer_eval)
                proof_data.append(f"last_layer_const:{self.last_layer_const}")
                break
    
    @staticmethod
    def next_eval(curr_eval, beta):
        coset_dilution = int(g_order / len(curr_eval))
        new_eval = []
        for i in range(len(curr_eval)/2):
            plus_part = (curr_eval[i] + curr_eval[len(curr_eval)/2 + i]) / 2
            minus_part = (curr_eval[i] - curr_eval[len(curr_eval)/2 + i]) / (2 * w**coset_dilution * g**(coset_dilution * i))
            new_eval.append(plus_part + beta * minus_part)
        
        # print(new_eval)
        # print("\n\n\n\nzzzz\n\n\n\n")
        return new_eval
        
    def query(self, fiat_shamir, proof_data):
        index = fiat_shamir.get_random_int(proof_data) % len(layers[0].data)
        negative_index = (index + len(layers[0].data)/2) % len(layers[0].data)
        for layer_index in len(layers):
            positive_data, positive_path = self.layers[layer_index].get_path(int(index / (2 ** layer_index)))
            negative_data, negative_path = self.layers[layer_index].get_path(int(negative_index/ (2 ** layer_index)))
            proof_data.append(f"layer_plus_eval:{positive_data}")
            proof_data.append(f"path:{positive_path}")
            proof_data.append(f"layer_minus_eval:{negative_data}")
            proof_data.append(f"path:{negative_path}")

In [41]:
my_eval = []
for i in range(g_order):
    my_eval.append(17 * w ** 4 * g**(4*i))

fri = Fri(cp_eval, fiat_shamir, proof_data)


774576236

240647268